<a href="https://www.kaggle.com/code/dostonur/baseline-1-permutations-dos?scriptVersionId=189169631" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# What is about ?

Preliminary draft

Baseline to decompose element into product of generators by ML methods. Permutation groups. 

    - Generate train set by random walks , i.e. pairs (g, k) 
    - Train model to preduct k from g
    - Use it as heuristics in any algorithm which finds global minima of distnace, e.g. Metpropolis
    
    



# Preliminaries Install/Import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import time
t0start = time.time() 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
i = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
#     if dirname == '/kaggle/input/growth-in-finite-groups/UT4_growth':
    for filename in filenames:
        i+=1
        if i<= 1:
            print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

/kaggle/input/growth-in-finite-groups/HeisenbergComputations.sage


In [2]:
import torch 
import math

if torch.cuda.is_available():
    device = torch.device("cuda")
    dtype = torch.float
else:
    device = torch.device("cpu")
    dtype = torch.float#int16  
    dtype = torch.int16  
    
print(dtype)    
print(device)


torch.int16
cpu


# Preliminaries Groups data 

## Load moves data 

In [3]:
import numpy as np
import pandas as pd 
# fn = '/kaggle/input/santa-2023/puzzle_info.csv'
fn = '/kaggle/input/growth-in-finite-groups/puzzle_info.csv'
di = pd.read_csv(fn)
di['n_generators'] = di['allowed_moves'].apply(lambda x: len(x.split(':'))-1)
di['vector_len'] = di['allowed_moves'].apply(lambda x: len(x.split(']')[0].split('[')[1].split(',')))
di['puzzle_type_brief'] = di['puzzle_type'].apply(lambda x: x.split('_')[0])

display(di)
di.to_csv('moves.csv')

,puzzle_type,allowed_moves,n_generators,vector_len,puzzle_type_brief
0,cube_2/2/2,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,...",6,24,cube
1,cube_3/3/3,"{'f0': [0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, ...",9,54,cube
2,cube_4/4/4,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...",12,96,cube
3,cube_5/5/5,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...",15,150,cube
4,cube_6/6/6,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...",18,216,cube
5,cube_7/7/7,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...",21,294,cube
6,cube_8/8/8,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...",24,384,cube
7,cube_9/9/9,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...",27,486,cube
8,cube_10/10/10,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...",30,600,cube
9,cube_19/19/19,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...",57,2166,cube


## Choose example (e.g. 2x2x2 or 3x3x3 cube or etc )

In [4]:
group_gens_id ='cube_2/2/2'
group_gens_id ='cube_3/3/3'

m = di['puzzle_type'] == group_gens_id
allowed_moves = di['allowed_moves'][m].iat[0]
allowed_moves = str( allowed_moves ).replace("'", '"')
import json
dict_allowed_moves = json.loads(allowed_moves)
print(str(dict_allowed_moves)[:500] )
vec_len = len( dict_allowed_moves[list(dict_allowed_moves.keys())[0]])
array_allowed_moves = np.zeros(  (len(dict_allowed_moves),  vec_len), dtype = int )
for i in range(len(dict_allowed_moves)):
    array_allowed_moves[i,:] = dict_allowed_moves[list(dict_allowed_moves.keys())[i]]
array_allowed_moves    

list_generators = list(dict_allowed_moves.values())
print()
print(str(list_generators)[:500])

n_gens = len(list_generators)
print('number of generators:', n_gens)

{'f0': [0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, 9, 16, 13, 10, 17, 14, 11, 6, 19, 20, 7, 22, 23, 8, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 45, 39, 40, 46, 42, 43, 47, 24, 21, 18, 48, 49, 50, 51, 52, 53], 'f1': [0, 1, 2, 43, 40, 37, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 3, 20, 21, 4, 23, 24, 5, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 48, 38, 39, 49, 41, 42, 50, 44, 45, 46, 47, 25, 22, 19, 51, 52, 53], 'f2': [42, 39, 36, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,

[[0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, 9, 16, 13, 10, 17, 14, 11, 6, 19, 20, 7, 22, 23, 8, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 45, 39, 40, 46, 42, 43, 47, 24, 21, 18, 48, 49, 50, 51, 52, 53], [0, 1, 2, 43, 40, 37, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 3, 20, 21, 4, 23, 24, 5, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 48, 38, 39, 49, 41, 42, 50, 44, 45, 46, 47, 25, 22, 19, 51, 52, 53], [42, 39, 36, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 0, 21, 

In [5]:
def inverse_permutation(perm):
    # Create an empty list to hold the inverse permutation
    inverse = [0] * len(perm)
    
    # Iterate over the original permutation
    for i, p in enumerate(perm):
        # Place the index at the correct position in the inverse permutation
        inverse[p] = i
    
    return inverse

list_generators_add_inverses = []
for perm in list_generators:
    perm_inv = inverse_permutation(perm)
    list_generators_add_inverses.append(perm)
    if perm_inv != perm:
        list_generators_add_inverses.append(perm_inv)
    else:
        print('perm == perm_inv - skipped')
        print(perm)        
        print(perm_inv)
        
list_generators =   list_generators_add_inverses     
print('List updated with inverses:')
print(str(list_generators)[:500])

list_generators_clean_duplicates = []
for p1 in list_generators:
    flag_already_in = False
    for p in list_generators_clean_duplicates:
        if p1 == p : 
            flag_already_in = True
            break
    if not flag_already_in :
        list_generators_clean_duplicates.append(p1)
        
list_generators = list_generators_clean_duplicates

n_gens = len(list_generators)       
print('number of generators:', n_gens)

List updated with inverses:
[[0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, 9, 16, 13, 10, 17, 14, 11, 6, 19, 20, 7, 22, 23, 8, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 45, 39, 40, 46, 42, 43, 47, 24, 21, 18, 48, 49, 50, 51, 52, 53], [0, 1, 2, 3, 4, 5, 18, 21, 24, 11, 14, 17, 10, 13, 16, 9, 12, 15, 47, 19, 20, 46, 22, 23, 45, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 8, 39, 40, 7, 42, 43, 6, 38, 41, 44, 48, 49, 50, 51, 52, 53], [0, 1, 2, 43, 40, 37, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 3, 20, 21, 4,
number of generators: 18


In [6]:
%%time
state_size = len(list_generators[0])
print('state_size:',state_size)
all_moves = torch.zeros( n_gens, state_size ,  device=device, dtype = torch.int32)
for k in range(n_gens):
    all_moves[k,:] = torch.tensor( list_generators[k],  device=device, dtype = torch.int32)
all_moves

state_size: 54
CPU times: user 4.9 ms, sys: 4.46 ms, total: 9.37 ms
Wall time: 47.4 ms


tensor([[ 0,  1,  2,  3,  4,  5, 44, 41, 38, 15, 12,  9, 16, 13, 10, 17, 14, 11,
          6, 19, 20,  7, 22, 23,  8, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 45, 39, 40, 46, 42, 43, 47, 24, 21, 18, 48, 49, 50, 51, 52, 53],
        [ 0,  1,  2,  3,  4,  5, 18, 21, 24, 11, 14, 17, 10, 13, 16,  9, 12, 15,
         47, 19, 20, 46, 22, 23, 45, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37,  8, 39, 40,  7, 42, 43,  6, 38, 41, 44, 48, 49, 50, 51, 52, 53],
        [ 0,  1,  2, 43, 40, 37,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18,  3, 20, 21,  4, 23, 24,  5, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 48, 38, 39, 49, 41, 42, 50, 44, 45, 46, 47, 25, 22, 19, 51, 52, 53],
        [ 0,  1,  2, 19, 22, 25,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 50, 20, 21, 49, 23, 24, 48, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36,  5, 38, 39,  4, 41, 42,  3, 44, 45, 46, 47, 37, 40, 43, 51, 52, 53],
        [42, 39, 36,  3,

# Train set generation

## Function to generate

In [7]:
%%time

def random_walks(generators , n_random_walk_length,  n_random_walks_to_generate,    rw_start = None, 
                 device=device, dtype = dtype , verbose = 0   ):
    '''
    Output:
    returns X,y: X - array of states, y - number of steps rw achieves it
    
    Input: 
    generators - generators (moves) to make random walks  (permutations), 
        can be list of vectors or array with vstacked vectors
    n_random_walk_length - number of visited nodes, i.e. number of steps + 1 
    n_random_walks_to_generate - how many random walks will run in parrallel
    rw_start - initial states for random walks - by default we will use 0,1,2,3 ...
        Can be vector or array
        If it is vector it will be broadcasted n_random_walks_to_generate times, 
        If it is array n_random_walks_to_generate - input n_random_walks_to_generate will be ignored
            and will be assigned: n_random_walks_to_generate = rw_start.shape[0]
    
    '''

    # Prepare generators in tensor format:
    if isinstance(generators, list):
        state_size = len(generators[0])
        all_moves = torch.zeros( n_gens, state_size ,  device=device, dtype = torch.int32) # Cannot use signed types 
        for k in range(n_gens):
            all_moves[k,:] = torch.tensor( generators[k],  device=device, dtype = torch.int32) 
    else:
        all_moves = generators
        state_size = generators.shape[1]
    n_generators = all_moves.shape[0] 
    
    if verbose >= 100:
        print('state_size', state_size )
        

    # Initialize current array of steps with  state_rw_start - expanding (broadcasting) it into array: 
    if rw_start is None:        
        # Starting state for random walks (for train data it is our destination state - "solved puzzle state")  
        state_rw_start = torch.arange( state_size, device=device, dtype = dtype)
        array_of_states = state_rw_start.view(1, state_size  ).expand(n_random_walks_to_generate, state_size )
    else:
        if len(rw_start.shape) == 1:
            array_of_states = rw_start.view(1, state_size  ).expand(n_random_walks_to_generate, state_size )
        else :
            array_of_states = rw_start
    if verbose >= 100:
        print('state_rw_start.shape:', state_rw_start.shape)
        print('state_rw_start:',state_rw_start)
        print(array_of_states.shape)
        print( array_of_states[:3,:] )

    # Output: X,y - states, y - how many steps we achieve them 
    # Allocate memory: 
    X = torch.zeros( (n_random_walks_to_generate)*n_random_walk_length , state_size,    device=device, dtype = dtype )
    y = torch.zeros( (n_random_walks_to_generate)*n_random_walk_length ,    device=device, dtype = dtype )
    if verbose >= 100:
        print('X.shape',X.shape)

    # First portion of data  - just our state_rw_start state  multiplexed many times
    X[:n_random_walks_to_generate,:] = array_of_states
    y[:n_random_walks_to_generate] = 0

    # Technical to make array[ IX_array] we need  actually to write array[ range(N), IX_array  ]
    row_indices = torch.arange( array_of_states.shape[0] , device=device, dtype = dtype )
    row_indices = np.arange( array_of_states.shape[0] )[:, np.newaxis]

    # Main loop 
    for i_step in range(1,n_random_walk_length):
        y[ (i_step)*n_random_walks_to_generate : (i_step+1)*n_random_walks_to_generate  ] = i_step
        IX_moves = np.random.randint(0, n_generators, size=n_random_walks_to_generate, dtype = int) # random moves indixes
        new_array_of_states = array_of_states[ row_indices , all_moves[IX_moves,:]] # all_moves[IX_moves,:] ] 
        array_of_states = new_array_of_states
        X[ (i_step)*n_random_walks_to_generate : (i_step+1)*n_random_walks_to_generate , : ] = new_array_of_states

    if verbose >= 100:
        print(array_of_states.shape, 'array_of_states.shape')
        print(n_random_walk_length, 'n_random_walk_length', state_size,'state_size', '' )    
        print('Finished')
        print(str(X)[:500] )
        print(str(y)[:500] )
        
    return X,y

X_tmp, y_tmp = random_walks(list_generators , n_random_walk_length = 3,  n_random_walks_to_generate =  2) #,    rw_start = None, 
                 #device=device, dtype = dtype , verbose = 0   )

print('X:')
print('X.shape', X_tmp.shape)
print(X_tmp)

print('y')
print('y.shape', y_tmp.shape)
print(y_tmp)

X:
X.shape torch.Size([6, 54])
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53],
        [ 0,  1,  2,  3,  4,  5, 44, 41, 38, 15, 12,  9, 16, 13, 10, 17, 14, 11,
          6, 19, 20,  7, 22, 23,  8, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 45, 39, 40, 46, 42, 43, 47, 24, 21, 18, 48, 49, 50, 51, 52, 53],
        [ 9,  1,  2, 12,  4,  5, 15,  7,  8, 45, 10, 11, 48, 13, 14, 51, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,  6, 30, 31,  3, 33, 34,  0,
         38, 41, 44, 37, 40, 43, 36, 39, 42, 35, 46, 47, 32, 49, 50, 29, 52

## Genration

In [8]:
X_train, y_train = random_walks(list_generators ,n_random_walk_length = 30 ,  n_random_walks_to_generate = 10_000 ) #,    rw_start = None, 
                 #device=device, dtype = dtype , verbose = 0   )

X_test, y_test = random_walks(list_generators ,n_random_walk_length = 30 ,  n_random_walks_to_generate = 10_000 ) #,    rw_start = None, 
                 #device=device, dtype = dtype , verbose = 0   )
    

# Modeling

In [9]:
%%time
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


model = CatBoostRegressor(depth=10, l2_leaf_reg=3, verbose=False)
str_model = 'CatBoost'

CPU times: user 264 ms, sys: 49.2 ms, total: 314 ms
Wall time: 675 ms


In [ ]:
#changed parameter depth and tried several options like 4, 6, 8, 10. The most suitable one was 4

In [10]:
X_train, y_train = X_train.numpy(), y_train.numpy()
X_test, y_test = X_test.numpy(), y_test.numpy()

In [11]:
%%time

df_stat = pd.DataFrame()

t0 = time.time()
model.fit(X_train, y_train)
fit_time = time.time() - t0

t0 = time.time()
y_pred = model.predict(X_test)
predict_time = time.time() - t0

t0 = time.time()
y_pred = model.predict(X_test)
predict_time = time.time() - t0
y_pred_train = model.predict(X_train)
r2_test = r2_score(y_test, y_pred)
rmse_test = mean_squared_error(y_test, y_pred,  squared=False)

r2_train = r2_score(y_train, y_pred_train)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)


IX = len(df_stat)+1

df_stat.loc[IX,'Model'] = str_model
df_stat.loc[IX,'r2 Test'] = r2_test
df_stat.loc[IX,'r2 Train'] = r2_train
df_stat.loc[IX,'rmse Test'] = rmse_test
df_stat.loc[IX,'rmse Train'] = rmse_train
df_stat.loc[IX,'FitTime'] = np.round(fit_time,3)
df_stat.loc[IX,'PredictTime'] = np.round(predict_time,3)
df_stat.loc[IX,'Train'] = str(X_train.shape)
df_stat.loc[IX,'Test'] = str(X_test.shape)



df_stat.round(3)

CPU times: user 16min 52s, sys: 10 s, total: 17min 2s
Wall time: 4min 56s


,Model,r2 Test,r2 Train,rmse Test,rmse Train,FitTime,PredictTime,Train,Test
1,CatBoost,0.723,0.794,4.553,3.927,266.777,9.731,"(300000, 54)","(300000, 54)"


# Pathfinder by Metropolis + ML model



In [ ]:
%%time

n_generators = len(list_generators)

print('state_size', state_size )

# Params:
n_walks_to_generate = 1_000
n_walk_length = 20 # _000#
n_trials = 10
beta = -0.5 # Inverse to temperature - when zero it will be pure random walk
n_scrambles_starting_state = 2
print('n_scrambles_starting_state:',n_scrambles_starting_state, 'beta:',beta, 'n_walks_to_generate', n_walks_to_generate, 'n_walk_length',n_walk_length , 'n_trials', n_trials)


# destination state - "solved puzzle state"   
state_destination = torch.arange( state_size, device=device, dtype = dtype)


# Scramble - generate state which will be start of Metropolis   
state_start = state_destination
for k in range(n_scrambles_starting_state):
    IX_move = np.random.randint(0, n_gens, dtype = int) # random moves indixes
    state_start = state_start[ list_generators[IX_move]] # all_moves[IX_moves,:] ] 
    


# Technical to make array[ IX_array] we need  actually to write array[ range(N), IX_array  ]
row_indices = torch.arange( n_walks_to_generate , device=device, dtype = dtype )
row_indices = np.arange( n_walks_to_generate )[:, np.newaxis]


# Collect simulation statistics here 
df_stat = pd.DataFrame()

# Main loop 
for i_trial in range(n_trials):


    # Initialize current array of steps with  state_rw_start - expanding (broadcasting) it into array: 
    array_of_states = state_start.view(1, state_size  ).expand(n_walks_to_generate, state_size ).clone()
    #array_of_states = array_of_states.numpy()


    # storage for all moves , we will need them to restore path to destination if find it 
    all_IX_moves = torch.zeros( n_walks_to_generate, n_walk_length, device=device, dtype = int)
    
    t01 = time.time()
    flag_found_destination = False
    for i_step in range(1,n_walk_length):
        # Generate and make random moves: 
        #IX_moves = torch.randint(0, n_generators,  size= (n_walks_to_generate,),  device=device, dtype = int) # random moves indices
        IX_moves = np.random.randint(0, n_generators,  size= (n_walks_to_generate,))# ,  device=device, dtype = int) # random moves indices
        new_array_of_states = array_of_states[ row_indices , all_moves[IX_moves,:]] # all_moves[IX_moves,:] ] 

        # Energy for old states:
        # hamming_distance =  ((array_of_states - state_destination).to(torch.bool)).sum(axis = 1)
        # initial_energy = hamming_distance.clone()
        initial_energy = model.predict(array_of_states.numpy())
        initial_energy[initial_energy<=0]=0

        # Energy for new states 
        # hamming_distance =  ((new_array_of_states - state_destination).to(torch.bool)).sum(axis = 1)
        # proposed_energy = hamming_distance
        proposed_energy = model.predict(new_array_of_states.numpy())
        proposed_energy[proposed_energy<=0]=0

        # Calculate the probability of acceptance  # For beta = 0 it will be all 1
        #acceptance_prob = torch.exp(beta * (initial_energy - proposed_energy))
        acceptance_prob = np.exp(beta * (initial_energy - proposed_energy))

        # Generate random numbers for decision making
        #random_numbers = torch.rand(n_walks_to_generate, device=device, dtype=torch.float32)
        random_numbers = np.random.rand(n_walks_to_generate)#, device=device, dtype=torch.float32)

        # Accept or reject proposed states
        accept = random_numbers < acceptance_prob
        # For beta = 0 , acceptance_prob = 1 , so always accepted - so just random walk 

        # Change states and save steps
        all_IX_moves[accept,i_step] = torch.tensor(IX_moves[accept])
        array_of_states[accept,:] = new_array_of_states[accept,:]    

        # Check destination state found 
        sm = (proposed_energy == 0).sum().item()
        if  sm > 0:
            flag_found_destination = True
            print('Found destination state. ', 'i_step:', i_step, ' n_ways:', sm)
            # Statistics: 
            IX_tmp = len(df_stat)+1
            df_stat.loc[IX_tmp,'trial'] = i_trial
            df_stat.loc[IX_tmp,'Path Len'] = i_step
            df_stat.loc[IX_tmp,'n_paths'] = sm
            df_stat.loc[IX_tmp,'Time'] = np.round( time.time() - t01 , 2)
            break
    
print('Finished')
display(df_stat)
df_stat.to_csv('df_stat.csv')
if len(df_stat)>0:
    display(df_stat.describe() )
print('n_scrambles_starting_state:',n_scrambles_starting_state, 'beta:',beta, 'n_walks_to_generate', n_walks_to_generate, 'n_walk_length',n_walk_length , 'n_trials', n_trials)


# Final timing

In [ ]:
print('%.1f seconds passed total '%(time.time()-t0start) )
print('%.1f minutes passed total '%( (time.time()-t0start)/60)  )
print('%.2f hours passed total '%( (time.time()-t0start)/3600)  )